#  Per Python ad astra: interactive Astrodynamics with poliastro

![poliastro](img/logo_text.svg)

### Juan Luis Cano - 2022-07-13 @ SciPy US 2022

Duration: 2:35 PM to 3:05 PM = 30 minutes = 25 minutes talk + 5 minutes Q&A

This talk presents poliastro, an open-source Python library for interactive Astrodynamics that features an easy-to-use API and tools for quick visualization [Cano Rodríguez et al, 2016]. poliastro implements core Astrodynamics algorithms (such as the resolution of the Kepler and Lambert problems) and leverages numba, a Just-in-Time compiler for scientific Python, to optimize the running time. Thanks to Astropy, poliastro can perform seamless coordinate frame conversions and use proper physical units and timescales [The Astropy Collaboration, 2018]. At the moment, poliastro is the longest-lived Python library for Astrodynamics, has contributors from all around the world, and several New Space companies and people in academia use it.

During the talk, we will describe the two-layer architecture that allows poliastro to offer an approachable API with good performance, discuss the challenges we faced to validate our code, and comment on the successes and failures of the project in trying to build a rich and diverse community.

1. intro + astrodynamics 5 minutes
2. intro to poliastro code 5 minutes = 10
3. validation 5 minutes = 15
4. performance & api design 5 minutes = 20
5. community 5 minutes = 25

# Outline

1. Introduction
2. poliastro
3. Challenges
  - Validation
  - Performance & API design
  - Community building
4. How to contribute

# Who am I?

- **Aerospace Engineer** with a passion for orbits 🛰
- **Data Scientist Advocate** at **Orchest**, an open source pipeline orchestrator 🥑
  - Previously Developer Advocate at Read the Docs and Mission Planning & Execution Engineer at Satellogic 🌍
- Former core developer of **Astropy** and contributor to NumPy, SciPy, and others
- Organizer of the **PyData Madrid** monthly meetup (ex Python España, ex PyCon Spain) 🐍
- Hard Rock lover 🎸

Follow me! https://github.com/astrojuanlu/

![Me!](img/juanlu_esa.jpg)

# Quick intro to Orbital Mechanics

### Physics → Mechanics → Celestial Mechanics → Orbital Mechanics

> A branch of Mechanics (itself a branch of Physics) that studies practical problems regarding the motion of rockets and other human-made objects through space

## But why do bodies orbit?

<img src="img/newtons-cannonball.jpg" alt="Newton" width="400" ></img>

## If it's so simple, why all the fuss?

- Difficult to measure: these things move at around ~8 000 meters per second (**Austin-Dallas in 40 seconds**) and GPS precision is not that good
- ...But great accuracy is required: we want to take pictures of specific places from 700 kilometers distance!
- Many perturbations: the Earth is not a sphere, the Moon is very close, the sunlight pushes the satellite (yes!)...
- If you lose contact with the satellite, it's a needle in a haystack

![Debris](img/debris.gif)

# poliastro

<img src="img/logo_text.svg" alt="poliastro" width="500" style="float:right"></img>

> poliastro is a fast, easy to use Python library for interactive astrodynamics and orbital mechanics.

- **Pure Python**, accelerated with **numba**
- **MIT license** (permissive)
- Physical units, astronomical scales and more, thanks to **Astropy**
- Conversion between several orbit representations
- Analytical and numerical propagation
- Cool documentation 🚀 https://docs.poliastro.space/
- Version 0.17.0 released last week https://docs.poliastro.space/en/0.17.x/changelog.html#poliastro-0-17-0-2022-07-10

<div style="clear:both"></div>

## Brief history

- *2013*: First version: Octave + FORTRAN + Python
- *2014*: Refactor of the API, much friendlier
- *2015*: Replace FORTRAN algorithms by Python + numba 🚀
- *2016*: Izzo algorithm (Lambert's problem), 6th ICATT @ ESA
- *2017*: Summer of Code in Space (SOCIS), **OpenAstronomy & Astropy membership**, 1st OSCW @ ESOC
- *2018*: **Google Summer of Code (GSOC)**, #PyAstro18 @ Simons Fndn, expansion into the industry
- *2019*: Two students on GSOC, #PyAstro19 @ STScI + 3rd OSCW @ Athens, **Jorge joins the team**!
- *2020*: Another GSOC, **NumFOCUS affiliation**, first NumFOCUS Small Development Grant
- *2021*: More GSOC and NumFOCUS SDG, weekly community calls, usage in the Global Trajectory Optimization Competition
- *2022*: **SciPy US 2022**
- *2023*: 10 year anniversary! Version 1.0 ❓

## Orbits

In [1]:
# https://github.com/plotly/plotly.py/issues/1664#issuecomment-511773518
import plotly.graph_objects as go
import plotly.io as pio

# Set default renderer
pio.renderers.default = "plotly_mimetype+notebook_connected+jupyterlab"

# Set default template
pio.templates["slides"] = go.layout.Template(layout=dict(width=700, height=550))
pio.templates.default = "plotly+slides"

In [2]:
from poliastro.examples import iss

iss

6772 x 6790 km x 51.6 deg (GCRS) orbit around Earth (♁) at epoch 2013-03-18 12:00:00.000 (UTC)

In [3]:
iss.plot(label="ISS", interactive=True)

In [4]:
iss.plot(label="ISS", interactive=True, use_3d=True)

In [5]:
from astropy import units as u
from astropy.time import Time

In [6]:
from poliastro.bodies import Earth
from poliastro.twobody import Orbit

In [7]:
r = [-6045, -3490, 2500] << u.km
v = [-3.457, 6.618, 2.533] << (u.km / u.s)

orb = Orbit.from_vectors(Earth, r, v, Time.now())
orb

7283 x 10293 km x 153.2 deg (GCRS) orbit around Earth (♁) at epoch 2022-07-11 16:55:01.162448 (UTC)

In [8]:
Orbit.from_classical(
    Earth,
    a=42000 << u.km,
    ecc=0.01 << u.one,
    inc=0 << u.deg,
    raan=0 << u.deg,
    argp=0 << u.deg,
    nu=0 << u.deg,
    epoch=Time.now(),
)

41580 x 42420 km x 0.0 deg (GCRS) orbit around Earth (♁) at epoch 2022-07-11 16:55:01.185135 (UTC)

## Propagation

In [9]:
iss.nu.to(u.deg), iss.epoch

(<Quantity 46.59580468 deg>,
 <Time object: scale='utc' format='iso' value=2013-03-18 12:00:00.000>)

In [10]:
iss_future = iss.propagate(30 << u.min)  # Returns a new Orbit!
iss_future.nu.to(u.deg), iss_future.epoch

(<Quantity 163.1409362 deg>,
 <Time object: scale='utc' format='iso' value=2013-03-18 12:30:00.000>)

`Ephem` objects can hold the time history of the propagation:

In [11]:
iss_ephem = iss.to_ephem()
iss_ephem

Ephemerides at 100 epochs from 2013-03-18 12:23:55.155 (UTC) to 2013-03-18 13:56:32.125 (UTC)

In [12]:
iss_ephem.sample()[:5]

<CartesianRepresentation (x, y, z) in km
    [( 859.07256   , -4137.20368   , 5295.56871   ),
     (1270.55257535, -4012.16848983, 5309.55706958),
     (1676.93829596, -3870.95571409, 5302.1480373 ),
     (2076.59334996, -3714.13396678, 5273.37144672),
     (2467.9084675 , -3542.33471373, 5223.34317102)]
 (has differentials w.r.t.: 's')>

## Other features

- Building `Orbit` and `Ephem` objects from public APIs
  - SPICE kernels (Astropy), SBDB & HORIZONS (astroquery)
- Analytical propagation, aka Kepler's problem (8 different algorithms)
- Numerical propagation (Cowell's method)
- Boundary-value problem, aka Lambert's problem (Vallado, Izzo with multiple revolutions)
- Orbital maneuvers (Hohmann's & bielliptic transfers, single impulse)
- Continuous thrust guidance laws (4 different guidance laws)

In [13]:
import matplotlib.pyplot as plt

from poliastro.plotting.misc import plot_solar_system
from poliastro.ephem import Ephem
from poliastro.frames import Planes
from poliastro.util import time_range

from astropy.coordinates import solar_system_ephemeris
solar_system_ephemeris.set("jpl")

<ScienceState solar_system_ephemeris: 'jpl'>

In [14]:
florence_orbit = Orbit.from_sbdb("Florence")

# Ephem.from_horizons("1P", Time.now().tdb)  # Fails, because there are several observations
halley_1835_ephem = Ephem.from_horizons(
    "90000028",
    time_range(Time("1835-06-01", scale="tt"), end=Time("1837-01-01", scale="tt")),
    plane=Planes.EARTH_ECLIPTIC,
)

plotter = plot_solar_system(epoch=Time.now().tdb, outer=False, interactive=True)

plotter.plot_ephem(halley_1835_ephem, label='Halley trajectory (ca. 1835)', color='#84b0b8')
plotter.plot(florence_orbit, label='Florence orbit', color='#000000')

In [15]:
import numpy as np

from numba import njit

from poliastro.core.propagation import func_twobody
from poliastro.twobody.propagation import CowellPropagator
from poliastro.twobody.sampling import EpochsArray
from poliastro.plotting import OrbitPlotter3D


@njit
def accel(t0, state, k):
    """Constant acceleration aligned with the velocity. """
    v_vec = state[3:]
    norm_v = (v_vec * v_vec).sum() ** .5
    return 1e-5 * v_vec / norm_v


@njit
def f(t0, u_, k):
    du_kep = func_twobody(t0, u_, k)  # Keplerian acceleration
    ax, ay, az = accel(t0, u_, k)  # Perturbation acceleration
    du_ad = np.array([0, 0, 0, ax, ay, az])  # Assemble state vector
    return du_kep + du_ad  # Add perturbation acceleration components

In [16]:
plotter = OrbitPlotter3D()

plotter.set_attractor(iss.attractor)
plotter.plot_ephem(iss.to_ephem(
    EpochsArray(
        time_range(iss.epoch, end=iss.epoch + (3 << u.day), periods=5_000),
        method=CowellPropagator(f=f),
    )
))

# Challenges

## Validation

Unit testing a function with clear expectations is trivial. What are our expectations on numerical algorithms?

The wrooooooooooooooong way:

In [17]:
import ipytest
ipytest.autoconfig(addopts=['-q', '--assert=plain', '--color=yes'])

import pytest

In [18]:
def sinc(x):
    return np.sin(x) / x

In [19]:
%%ipytest

@pytest.mark.parametrize("x", [0, 1, 10])
def test_sinc(x):
    assert sinc(x) == np.sin(x) / x

F..                                                                                          [100%]
============================================= FAILURES =============================================
___________________________________________ test_sinc[0] ___________________________________________

x = 0

    @pytest.mark.parametrize("x", [0, 1, 10])
    def test_sinc(x):
>       assert sinc(x) == np.sin(x) / x
E       AssertionError: assert nan == (0.0 / 0)
E        +  where nan = sinc(0)
E        +  and   0.0 = <ufunc 'sin'>(0)
E        +    where <ufunc 'sin'> = np.sin

/tmp/ipykernel_42328/1906346432.py:3: AssertionError
========================================= warnings summary =========================================
tmp4zh3jac2.py::test_sinc[0]
  /tmp/ipykernel_42328/404149208.py:2: RuntimeWarning:
  
  invalid value encountered in double_scalars

tmp4zh3jac2.py::test_sinc[0]
  /tmp/ipykernel_42328/1906346432.py:3: RuntimeWarning:
  
  invalid value encountered in double_sca

Problems:

- Floating point equality: bad idea!
- Invites copy-pasting implementation into test, with its bugs and typos!

A better way:

* Compare against some authoritative source: **external data or software**
* Do floating point comparisons right and **use tolerances**
* Leverage advance features such as pytest **fixtures** and automatic test generation with hypotheses https://github.com/HypothesisWorks/hypothesis/

In [20]:
from poliastro.twobody.states import ClassicalState
from astropy.tests.helper import assert_quantity_allclose

In [21]:
%%ipytest
def test_convert_from_rv_to_coe():
    # Data from Vallado, example 2.6
    attractor = Earth
    p = 11067.790 << u.km
    ecc = 0.83285 << u.one
    inc = 87.87 << u.deg
    raan = 227.89 << u.deg
    argp = 53.38 << u.deg
    nu = 92.335 << u.deg
    expected_r = [6525.344, 6861.535, 6449.125] << u.km
    expected_v = [4.902276, 5.533124, -1.975709] << (u.km / u.s)

    r, v = ClassicalState(attractor, (p, ecc, inc, raan, argp, nu), None).to_vectors().to_tuple()

    assert_quantity_allclose(r, expected_r, rtol=1e-5)
    assert_quantity_allclose(v, expected_v, rtol=1e-5)

.                                                                                            [100%]
1 passed in 0.00s


Use property-based testing:

In [22]:
from functools import partial

from hypothesis import example, given, settings, strategies as st
import numpy as np

angles = partial(st.floats, min_value=-2 * np.pi, max_value=2 * np.pi)
eccentricities = partial(st.floats, min_value=0, max_value=1, exclude_max=True)

@st.composite
def with_units(draw, elements, unit):
    angle = draw(elements)
    return angle * unit

angles_q = partial(with_units, elements=angles(), unit=u.rad)
eccentricities_q = partial(with_units, elements=eccentricities(), unit=u.one)

In [23]:
from poliastro.twobody.sampling import sample_closed

In [24]:
%%ipytest
@given(
    min_nu=angles_q(),
    ecc=eccentricities_q(),
    max_nu=st.one_of(angles_q(), st.none()),
)
def test_sample_closed_is_always_between_minus_pi_and_pi(min_nu, ecc, max_nu):
    result = sample_closed(ecc, min_nu, max_nu)

    assert ((-np.pi << u.rad <= result) & (result <= np.pi << u.rad)).all()

.                                                                                            [100%]
1 passed in 0.27s


Still some issues:

- How much precision do you ask for? Should you carry a mathematical analysis?
- What if your results don't match? Sometimes, book or paper authors respond to your comments... And sometimes don't
- The changes in precision are a result of bad data, or worse algorithms?
- How do you even track _improvements_?

### External data (short summary)

![Data "available upon (reasonable) request"](img/upon-request.png)

(https://twitter.com/ceptional/status/1533567322736435200)

![Shrug](img/shrugging-guy.jpg)

### External software

- Sometimes commercial
- Is it validated itself? -> "Validation by consensus"
- It is often difficult to reproduce the exact setting and algorithms, most of the times because your commercial software is much more complex

Despite these difficulties, we are validating poliastro against Orekit and GMAT! https://github.com/poliastro/validation/

More info:

- my Masters thesis https://github.com/astrojuanlu/pfc-uc3m
- Jorge's Bachelors thesis https://github.com/jorgepiloto/lamberthub/tree/main/art

## Performance and API design

- We want to be **as user friendly as possible**
- This includes protecting the user from common mistakes
- Two annoying sources of errors: physical units and reference frames

![Mars Climate Orbiter](img/mco.png)

- But performance comes at a price
- _Yes, Python is slow_ (compared to compiled languages)
- The places where we don't notice it is because the underlying code is compiled (e.g. NumPy)

Then, how to accelerate the code?

Several options:

- **Vectorization** (fancy name for "find a NumPy function that does what you need")
  - Not always possible or straightforward, vectorized code can difficult to read
- **Cython**
  - "Two language problem" poses barrier to contribute and install, also I don't know any C
- **PyPy**
  - Still not widely supported, Astropy is close but not there
- **numba**
  - Our tool of choice!

## numba

![numba](img/numba-logo.png)

* numba is a Python-to-LLVM JIT compiler
* When it works, it's super effective and the results are impressive!
* Debugging improved _a lot_ lately
* Passing functions as arguments has a bit of overhead https://github.com/numba/numba/issues/2952
* Its focus is numerical code: it won't accelerate all Python features

## Solution

So... let's make our code Fortran-esque!

<img src="img/architecture.svg" alt="Architecture" width="500" style="float:right"></img>

High level API:

* Supports mixed units and time scales, figures out the rest
* Easy to use and impossible to get wrong
* **Slower** (hopefully small overhead)

Dangerous™ algorithms:

* **Fast** (easy to accelerate with numba or Cython)
* Only cares about numbers, makes assumptions on units (SI, TBD)
* **You can mess it up**

<div style="clear:both"></div>

In [25]:
from poliastro.core.angles import E_to_nu as E_to_nu_fast


@u.quantity_input(E=u.rad, ecc=u.one)
def E_to_nu(E, ecc):
    """True anomaly from eccentric anomaly."""
    return (
        E_to_nu_fast(
            E.to_value(u.rad),
            ecc.value
        ) << u.rad
    ).to(E.unit)


E_to_nu(70 << u.deg, 0.24 << u.one)

<Quantity 83.61877258 deg>

### Measure everything!

https://benchmarks.poliastro.space/

![Benchmarks](img/benchmarks.png)

## Community building

### Licensing and engagement

> I believe the choice of license is an important one, and I advocate a BSD-style license. **In my experience, the most important commodity an open source project needs to succeed is users.**
>
> -- John Hunter † http://nipy.org/nipy/faq/johns_bsd_pitch.html

- Permissive licenses guarantee maximum adoption
- ...but sometimes this adoption is **invisible**
- Some effort is needed to establish a communication channel with users/stakeholders (we are failing at this)
- Should we have chosen copyleft instead? Impossible to debate this online

### Going beyond the code

- Sometimes, language wins over performance
  - "Python is the second best language for everything"
  - Would Julia be a better choice? Still key astronomical and visualization pieces missing
- Sometimes, documentation wins over features
  - A great library with bad documentation will be hated at best, ignored at worst
- Sometimes, marketing wins over quality
  - poliastro has a cute logo
  - And we talk about it at every Python event we go
  - However, we don't attend enough industry events!

# _Per Python ad astra!_ 🚀

* Slides: https://github.com/astrojuanlu/scipy-us-2022-poliastro-talk
* poliastro chat on Matrix: http://chat.poliastro.space
* Twitter: https://twitter.com/poliastro_py

## Thank you!

![Rocket](img/vega.jpg)